# ComCrop ONNX Model run on OpenEO backend

This notebook guides you through running the ComCrop ONNX model using OpenEO.
It allows you to interactively select the Area of Interest (AOI) on a map.

**Instructions:**
1. Ensure `openeo`, `rasterio`, `matplotlib`, `ipyleaflet` and `ipywidgets` are installed.
2. Run the 'Imports' cell.
3. Run the 'Define Area of Interest (AOI)' cell. A map will appear.
4. Use the rectangle drawing tool (on the left side of the map) to draw your desired AOI.
5. The coordinates and approximate size (in 10m pixels) will be displayed below the map.
   - **Constraint:** The maximum size allowed is 600x600 pixels. If your selection is too large, an error message will appear, and you'll need to draw a smaller rectangle.
6. Once a valid AOI is selected, proceed to the subsequent cells to connect to OpenEO and run the processing job.

In [ ]:
# --- Import Libraries ---
import openeo
import rasterio
from rasterio.plot import show
import ipyleaflet
from ipyleaflet import Map, DrawControl, Rectangle, GeoJSON
from ipywidgets import Output, VBox, HTML
from IPython.display import display
import datetime
import math
import json
import os
import warnings
import matplotlib.pyplot as plt


warnings.filterwarnings('ignore', category=FutureWarning) # Ignore specific FutureWarnings often seen with geo-libraries

## Define Area of Interest (AOI)

Use the map below to draw a rectangle defining your processing area. The maximum size is approximately 600x600 pixels at 10m resolution.

In [ ]:
# --- Select AOI on a map ---
import math
from ipyleaflet import Map, DrawControl
from ipywidgets import VBox, HTML, Output
import time
 
# --- Configuration ---
DEFAULT_RESOLUTION_M = 10
INITIAL_ZOOM = 12
DEFAULT_CENTER = (4.7, -74.1)  # Bogotá, Colombia
MAX_AREA_SQ_M = (600 * DEFAULT_RESOLUTION_M) ** 2
MAX_HECTARES = MAX_AREA_SQ_M / 10000  # 3600 ha

# --- Widgets ---
selection_status = HTML(value=f"Please draw a rectangle on the map (max area: {MAX_HECTARES:.2f} ha).")
output_widget = Output()

# --- State ---
selected_extent = {}

# --- Map Creation ---
m = Map(center=DEFAULT_CENTER, zoom=INITIAL_ZOOM, scroll_wheel_zoom=True)

# --- Draw Control ---
draw_control = DrawControl(
    rectangle={'shapeOptions': {'color': '#0078A8', 'fillOpacity': 0.1}},
    polygon={}, circlemarker={}, circle={}, polyline={}, marker={}
)
m.add_control(draw_control)

# --- Helper Functions ---
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def calculate_dimensions_and_area(west, south, east, north):
    mid_lat = (south + north) / 2
    width_m = calculate_distance(mid_lat, west, mid_lat, east)
    height_m = calculate_distance(south, west, north, west)
    area_sq_m = width_m * height_m
    area_ha = area_sq_m / 10000
    return width_m, height_m, area_ha

# --- Draw Callbacks ---
def on_draw_handler(event=None, action=None, geo_json=None, **kwargs):
    global selected_extent
    with output_widget:
        output_widget.clear_output()  # Clear previous output to show only the latest rectangle info
        if action == 'created' or action == 'modified':
            if geo_json:
                coords = geo_json['geometry']['coordinates'][0]
                if len(coords) == 5:  # Rectangle has 5 points (closed)
                    lons = [p[0] for p in coords]
                    lats = [p[1] for p in coords]
                    west, east = min(lons), max(lons)
                    south, north = min(lats), max(lats)

                    width_m, height_m, area_ha = calculate_dimensions_and_area(west, south, east, north)
                    width_px = width_m / DEFAULT_RESOLUTION_M
                    height_px = height_m / DEFAULT_RESOLUTION_M
                    
                    if area_ha > MAX_HECTARES:
                        selection_status.value = f"<font color='red'>Error: Area ({area_ha:.2f} ha) exceeds max allowed ({MAX_HECTARES:.2f} ha). Please draw a smaller rectangle.</font>"
                        return
                    
                    print(f"Valid rectangle drawn:")
                    print(f"   Extent: W: {west:.6f}, S: {south:.6f}, E: {east:.6f}, N: {north:.6f}")
                    print(f"   Size (approx @{DEFAULT_RESOLUTION_M}m): {width_px:.1f} x {height_px:.1f} pixels")
                    print(f"   Area (approx): {area_ha:.2f} ha")
                    
                    selected_extent = {
                        'west': west, 'south': south, 'east': east, 'north': north, 'crs': 'EPSG:4326'
                    }
                    selection_status.value = f"<font color='green'>Valid AOI selected ({area_ha:.2f} ha). You can now proceed to the next cell.</font>"
        elif action == 'deleted':
            selected_extent = {}
            selection_status.value = f"Please draw a rectangle on the map (max area: {MAX_HECTARES:.2f} ha)."

# --- Register Callbacks & Display ---
draw_control.on_draw(on_draw_handler)
display(VBox([selection_status, m, output_widget]))

## Connect to OpenEO Backend, Load Data, Run UDFs and Run ONNX Model on the Backend

In [ ]:
# --- OpenEO Connection ---
print("Connecting to openeo.dataspace.copernicus.eu...")
try:
    # Direct connection to Copernicus Data Space
    connection = openeo.connect("openeo.dataspace.copernicus.eu")
    
    # Simple authentication using CDSE provider
    print("Authenticating with CDSE provider...")
    connection.authenticate_oidc(provider_id='CDSE')
    print("Successfully authenticated.")
    
    # Print OpenEO client version for reference
    print(f"OpenEO Client Version: {openeo.__version__}")
    
except Exception as e:
    print(f"Error connecting or authenticating: {e}")
    connection = None  # Ensure connection is None if failed

In [ ]:
# --- Load Configuration ---

# Ensure connection is established before proceeding
if connection is None:
    raise ConnectionError("Failed to establish OpenEO connection in the previous step.")

# Check if a valid extent was selected
if not selected_extent:
     raise ValueError("No valid Area of Interest (AOI) was selected on the map. Please go back and draw a valid rectangle.")
else:
    print("Using AOI selected from the map:")
    print(json.dumps(selected_extent, indent=2))
    SPATIAL_EXTENT = selected_extent # Use the globally stored extent


# URL for model and other dependencies (adjust if needed)
model_url = "https://git.wur.nl/openeo_monitor/comcrop/-/raw/main/openeo_udf/jupyter_example/model.zip"

# Model filename inside the zip
model_filename = "comcrop_udf_test.onnx" # Make sure this matches the actual ONNX file in model.zip

# Path to the model UDF script (relative to notebook or absolute)
model_udf_path = "model_udf_onnx.py" # Assumes it's in the same directory as the notebook

# --- Check if UDF file exists ---
if not os.path.exists(model_udf_path):
     raise FileNotFoundError(f"Model UDF script not found at: {model_udf_path}")
else:
    print(f"Model UDF script found: {model_udf_path}")


# --- Define the UDF context ---
print("Defining the UDF context...")
model_udf_context = {
    "model_path": f"model/{model_filename}", # Path *inside* the OpenEO backend environment after archive extraction
}

# --- Configure job options ---
# Adjust memory based on typical job needs and selected area size
# Calculate approximate pixels based on selected extent if needed for dynamic sizing
# Ensure width_px and height_px exist in selected_extent
total_pixels = selected_extent.get('width_px', 600) * selected_extent.get('height_px', 600)

# Example of dynamic memory allocation (but needs refinement based on actual usage)
driver_mem = "4G" # Usually less needed for driver
executor_mem_base = 500 # MB Base memory for executor
executor_overhead_base = 4000 # MB Base overhead
# Heuristic: Add overhead based on pixel count (experiment: adjust factor as needed)
pixels_per_gb_overhead = 1_500_000 # e.g., 1 GB per 1.5M pixels
extra_overhead_gb = math.ceil(total_pixels / pixels_per_gb_overhead)
executor_overhead = f"{executor_overhead_base + extra_overhead_gb * 1024}m"

print(f"Estimated total pixels: {total_pixels:,.0f}")
print(f"Calculated executor memory overhead: {executor_overhead}")


print("Defining the job options...")
job_options = {
    "udf-dependency-archives": [
        f"{model_url}#model",  # Fetches zip, extracts to 'model' dir on backend
    ],
    "driver-memory": driver_mem,
    "executor-memory": f"{executor_mem_base}m",
    "executor-memoryOverhead": executor_overhead,
    "log_level": "info", # Use 'debug' for more verbose UDF logs if needed
}
print("Job Options:")
print(json.dumps(job_options, indent=2))


# --- Define Temporal Extent ---
# We might want to make this interactive using ipywidgets too (e.g., DatePickers)
# For now, using the dates from the original script
start_date = "2023-03-24"
end_date = "2023-03-24"
TEMPORAL_EXTENT = (start_date, end_date)

# SAR requires a wider range for meaningful mean calculation
sar_start_date = "2023-01-01"
sar_end_date = "2023-12-31"
SAR_TEMPORAL_EXTENT = (sar_start_date, sar_end_date)

print(f"Temporal Extent (Optical): {TEMPORAL_EXTENT}")
print(f"Temporal Extent (SAR): {SAR_TEMPORAL_EXTENT}")

In [ ]:
# --- Load UDFs ---

# Add coordinates via UDF - Define before using
add_coord_udf = openeo.UDF(f"""
import numpy as np
import xarray as xr
import math
import logging
from openeo.udf import XarrayDataCube

def apply_datacube(cube: XarrayDataCube, context: dict) -> XarrayDataCube:
    data = cube.get_array()

    west = {SPATIAL_EXTENT['west']}
    south = {SPATIAL_EXTENT['south']}
    east = {SPATIAL_EXTENT['east']}
    north = {SPATIAL_EXTENT['north']}
    
    logging.info("Coordinate UDF using extent: W:%.6f, S:%.6f, E:%.6f, N:%.6f" % (west, south, east, north))

    # Calculate pixel dimensions
    width_pixels = data.x.size
    height_pixels = data.y.size
    
    # Create evenly spaced arrays for longitude and latitude
    lon_array = np.linspace(west, east, width_pixels)
    lat_array = np.linspace(north, south, height_pixels)  # North to south for correct orientation

    # Create 2D meshgrid
    lon_grid, lat_grid = np.meshgrid(lon_array, lat_array)

    # Debug output
    logging.info("Extent: %.4f°W, %.4f°S, %.4f°E, %.4f°N" % (west, south, east, north))
    logging.info("Grid dimensions: %d×%d pixels" % (width_pixels, height_pixels))
    logging.info("Longitude range: %.6f to %.6f" % (west, east))
    logging.info("Latitude range: %.6f to %.6f" % (north, south))

    # Create dataset
    ds = xr.Dataset({{
        "lon": (("y", "x"), lon_grid),
        "lat": (("y", "x"), lat_grid)
    }}).to_array("bands")

    return XarrayDataCube(ds)
""")


print("Add Coordinates UDF defined.")


# Normalisation UDF - Define before using
normalise_bands_udf = openeo.UDF(f"""
import numpy as np
import xarray as xr
import logging
from openeo.udf import XarrayDataCube

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(message)s'  # Simple format showing only the message
)
logger = logging.getLogger(__name__)

def lnp(message):
    '''Log and print the message'''
    logger.info(message)

def normalise_vv(raster):
    raster = np.clip(raster, -25, 0)
    return (raster + 25) / 25

def normalise_vh(raster):
    raster = np.clip(raster, -30, -5)
    return (raster + 30) / 25

def normalise_longitude(raster):
    raster = np.clip(raster, -180, 180)
    return (raster + 180) / 360

def normalise_latitude(raster):
    raster = np.clip(raster, -60, 60)
    return (raster + 60) / 120

def normalise_altitude(raster):
    raster = np.clip(raster, -400, 8000)
    return (raster + 400) / 8400

def norm(image):
    NORM_PERCENTILES = np.array([
        [1.7417268007636313, 2.023298706048351],
        [1.7261204997060209, 2.038905204308012],
        [1.6798346251414997, 2.179592821212937],
        [2.3828939530384052, 2.7578332604178284],
        [1.7417268007636313, 2.023298706048351],
        [1.7417268007636313, 2.023298706048351],
        [1.7417268007636313, 2.023298706048351],
        [1.7417268007636313, 2.023298706048351],
        [1.7417268007636313, 2.023298706048351]
    ], dtype=np.float32)
    
    # Get the shape and ensure we're operating on the right dimensions
    if image.ndim == 3 and image.shape[0] == 9:
        # Image shape is (9, height, width)
        # Reshape percentiles for proper broadcasting: (9,1,1)
        min_values = NORM_PERCENTILES[:, 0].reshape(9, 1, 1)
        scale_values = NORM_PERCENTILES[:, 1].reshape(9, 1, 1)
    else:
        # Handle other shapes (original code path)
        min_values = NORM_PERCENTILES[:, 0]
        scale_values = NORM_PERCENTILES[:, 1]
    
    # Log the shapes for debugging
    # print(f"Image shape: {{image.shape}}, min_values shape: {{min_values.shape}}")
    
    image = np.log(image * 0.005 + 1)
    image = (image - min_values) / scale_values
    image = np.exp(image * 5 - 1)
    return (image / (image + 1)).astype(np.float32)

def apply_datacube(cube: XarrayDataCube, context: dict) -> XarrayDataCube:
    '''Normalise the input data cube for model inference.
    - Normalises the first 9 bands using the norm function
    - Normalises VV, VH, DEM, lon, lat separately
    - Calculates NDVI from B08 and B04
    - Outputs a 15-band cube
    '''
    original_data = cube.get_array()
    lnp(f"Received data with shape: {{original_data.shape}} and dims: {{original_data.dims}}")
    
    # Check if bands are already in the correct order
    if list(original_data.coords['bands'].values) != ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B11", "B12", "VV", "VH", "DEM", "lon", "lat"]:
        # Reorder if necessary
        expected_order = ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B11", "B12", "VV", "VH", "DEM", "lon", "lat"]
        lnp(f"Reordering bands from {{original_data.coords['bands'].values}} to {{expected_order}}")
        x_img = original_data.sel(bands=expected_order)
    else:
        # Already in correct order
        x_img = original_data
    
    # Get the band names from the cube
    band_names = x_img.coords['bands'].values
    
    # Validate input bands - we need all these bands in this exact order
    expected_bands = ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B11", "B12", "VV", "VH", "DEM", "lon", "lat"]
    for band in expected_bands:
        if band not in band_names:
            raise ValueError(f"Missing required band: {{band}}. Available bands: {{band_names}}")
    
    # Check if the order matches exactly
    if list(band_names) != expected_bands:
        # This check is now somewhat redundant due to reordering above, but kept for safety
        raise ValueError(f"Band order mismatch. Expected: {{expected_bands}}, Got: {{list(band_names)}}")
    
    # Get the indices for each band by name
    try:
        band_indices = {{band: list(band_names).index(band) for band in expected_bands}}
    except ValueError:
        raise ValueError(f"Could not find all required bands among {{band_names}}")
    
    # Convert to numpy for operations
    img_values = x_img.values
    
    # Normalize the first 9 bands (optical) as a single operation
    optical_bands = [img_values[band_indices[b]] for b in expected_bands[:9]]
    optical_bands_stack = np.stack(optical_bands, axis=0)
    normalised_optical = norm(optical_bands_stack)
    
    # Normalise individual bands
    vv_normalised = normalise_vv(img_values[band_indices["VV"]])
    vh_normalised = normalise_vh(img_values[band_indices["VH"]])
    dem_normalised = normalise_altitude(img_values[band_indices["DEM"]])
    lon_normalised = normalise_longitude(img_values[band_indices["lon"]])
    lat_normalised = normalise_latitude(img_values[band_indices["lat"]])
    
    # Calculate NDVI
    B08 = img_values[band_indices["B08"]]
    B04 = img_values[band_indices["B04"]]
    # Avoid division by zero
    denominator = B08 + B04
    ndvi = np.divide(B08 - B04, denominator, out=np.zeros_like(B08, dtype=np.float32), where=denominator!=0)
    ndvi_normalised = np.clip(ndvi, -1, 1) # Standard NDVI range
    
    # Stack all bands together: 9 optical + NDVI + VV + VH + DEM + Lon + Lat = 15 bands
    result_array = np.stack(
        [
            *normalised_optical,  # Unpack the 9 normalised optical bands
            ndvi_normalised,
            vv_normalised,
            vh_normalised,
            dem_normalised,
            lon_normalised,
            lat_normalised
        ],
        axis=0
    )
    
    # Create a new list of bands including NDVI - MUST HAVE EXACTLY 15 BANDS FOR MODEL
    new_bands = list(band_names[:9]) + ['NDVI'] + list(band_names[9:])
    
    # Double-check dimensions match
    if result_array.shape[0] != len(new_bands):
        raise ValueError(f"Band count mismatch: data shape {{result_array.shape[0]}} != label count {{len(new_bands)}}")    
    
    # Create output DataArray with the same coordinates but normalised values
    # Explicitly use (bands, y, x) order for GeoTrellis compatibility
    output_data = xr.DataArray(
        result_array,
        dims=['bands', 'y', 'x'],
        coords={{'bands': new_bands, 'y': x_img.coords['y'], 'x': x_img.coords['x']}}
    )
    
    lnp(f"Final output dimensions: {{output_data.dims}} with shape {{output_data.shape}}") 
    lnp(f"Final bands: {{output_data.coords['bands'].values}}") 
    
    return XarrayDataCube(output_data)

""")

print("Normalisation UDF defined.")


# Load the Model UDF from file
print(f"Loading Model UDF from file: {model_udf_path}...")

# Pass the context defined earlier (contains model_path for backend)
model_udf = openeo.UDF.from_file(model_udf_path, context=model_udf_context)
print("Model UDF loaded.")


In [ ]:
# --- Load Collections ---

# Check extent and connection again before loading
if not selected_extent or connection is None:
    raise ValueError("Cannot load data: OpenEO connection or spatial extent is missing.")

print("Loading Sentinel-1 collection (Global Mosaics)...")
# Load Data Collections
print("Loading Sentinel-1 collection...")
sentinel1 = connection.load_collection(
    "SENTINEL1_GLOBAL_MOSAICS",
    spatial_extent=SPATIAL_EXTENT,
    temporal_extent=SAR_TEMPORAL_EXTENT,
    bands=["VV", "VH"]
)
sar_cube = sentinel1.reduce_dimension(dimension="t", reducer="mean")

# Apply a simple dB conversion to the result
# Note: This is a simple log transform that assumes the data is in linear scale
print("Converting SAR data to dB scale (post time-reduction)...")
sar_cube = sar_cube.apply(lambda x: 10 * x.log(base=10))
print("SAR data converted to dB scale.")

print("Loading Sentinel-2 collection...")
sentinel2 = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=TEMPORAL_EXTENT,
    spatial_extent=SPATIAL_EXTENT,
    bands=["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B11", "B12"]
)

# Get RGB bands for resampling
sentinel2_RGB = sentinel2.filter_bands(["B04", "B03", "B02"])

print("Loading DEM collection...")
dem = connection.load_collection(
    "COPERNICUS_30",
    spatial_extent=SPATIAL_EXTENT
)


In [ ]:
# --- Resampling and Prepairing Collections ---
print("Resampling and preparing collections...")
# Reduce time dimension for all collections to eliminate temporal dimension issues
sentinel2_reduced = sentinel2.reduce_dimension(dimension="t", reducer="mean")
print("Sentinel-2 time dimension reduced.")

# Keep Sentinel-2 resampling to ensure all bands are properly aligned
sentinel2_resampled = sentinel2_reduced.resample_cube_spatial(sentinel2_RGB)
# Don't resample SAR explicitly - let merge_cubes handle it
sar_resampled = sar_cube.resample_cube_spatial(sentinel2_RGB, method="near")
# DEM needs bilinear resampling specifically
dem_resampled = dem.resample_cube_spatial(sentinel2_RGB, method="bilinear")

# Attempt to reduce time dimension for DEM (will be a no-op if dimension doesn't exist)
try:
    dem_resampled = dem_resampled.reduce_dimension(dimension="t", reducer="mean")
    print("DEM time dimension reduced.")
except Exception as e:
    print(f"Note: DEM doesn't have a time dimension or couldn't be reduced: {str(e)}")

print("Applying add-coordinate UDF...")
# Apply the coordinate UDF to SAR data with consistent resampling to ensure proper alignment
# Important: We need consistent resampling to avoid grid bounds errors
sar_resampled = sar_cube.resample_cube_spatial(sentinel2_RGB, method="near")
sentinel1_coords = sar_resampled.apply(process=add_coord_udf)
sentinel1_coords_resampled = sentinel1_coords.rename_labels("bands", ["lon", "lat"])

print("Collections resampled and prepared.")

# Merge data cubes
print("Merging data cubes...")
merged_datacube = (
    sentinel2_resampled
    .merge_cubes(sar_resampled)
    .merge_cubes(dem_resampled)
    .merge_cubes(sentinel1_coords_resampled)
)


# Preparing bands for model processing
print("Preparing bands for model processing...")
existing_band_labels = merged_datacube.dimension_labels('bands')
print(f"Original band labels: {existing_band_labels}")

# Rename bands to exactly match the expected order for the UDF
# No NDVI yet - it will be added by the normalization UDF
cubemerged = merged_datacube.rename_labels(
    'bands', ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B11", "B12", "VV", "VH", "DEM", "lon", "lat"]
)
print(f"Renamed band labels: {cubemerged.dimension_labels('bands')}")



In [ ]:
# --- Apply Normalisation UDF ---
print("Applying normalisation UDF...")
cubemerged = cubemerged.apply(process=normalise_bands_udf)

# After normalisation, we need to explicitly update band labels to include NDVI
# This ensures the dimension labels match the actual 15 bands output by the UDF
print("Updating band labels to include NDVI...")
cubemerged = cubemerged.rename_labels(
    'bands', ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B11", "B12", "NDVI", "VV", "VH", "DEM", "lon", "lat"]
)
print(f"Final band labels: {cubemerged.dimension_labels('bands')}")

In [ ]:
# --- Run Model UDF with Neighborhood Processing ---

print("Proceeding to apply_neighborhood...")

# Use smaller patches with overlap to prevent grid bounds issues
print("Running model UDF with neighborhood processing...")
patch_size = 48  # Reduced from 64 to avoid grid bounds issues
overlap = 8     # Add overlap to handle edge cases
print(f"Using patch size {patch_size}x{patch_size} with {overlap} pixel overlap")
print("Monitor batch job @ https://openeo.dataspace.copernicus.eu/")

result = cubemerged.apply_neighborhood(
    process=model_udf,
    size=[
        {'dimension': 'x', 'value': patch_size, 'unit': 'px'},
        {'dimension': 'y', 'value': patch_size, 'unit': 'px'}
    ],
    overlap=[
        {'dimension': 'x', 'value': overlap, 'unit': 'px'},
        {'dimension': 'y', 'value': overlap, 'unit': 'px'}
    ]
)

# Set up the output file path
target_file = f"{model_filename}~jn~prediction.tif"

# Create and start a batch job
main_job = result.execute_batch(target_file, 
    job_options=job_options, 
    title=f"{model_filename}~jn~ONNX Prediction"
)
print(f"Main batch job {main_job.job_id} finished. Output will be saved to {target_file}")


In [ ]:
# --- Visualise Results ---
if main_job.status() == 'finished':
    print("If job completed successfully, we can show results...")
    # Download results to the directory where the notebook is running
    results = main_job.get_results()
    results.download_files()
    print(f"Results (probably) downloaded. Check for file... {target_file}")
    
    
    # Now try to open and display the file
    print(f"Opening result file: {target_file}")
    with rasterio.open(target_file) as dataset:
        print(f"Dataset properties:")
        print(f"  Driver: {dataset.driver}")
        print(f"  CRS: {dataset.crs}")
        print(f"  Count: {dataset.count}")
        print(f"  Width: {dataset.width}, Height: {dataset.height}")
        print(f"  Bounds: {dataset.bounds}")

        # Display the first band
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        show(dataset.read(1), ax=ax, cmap='viridis', title=target_file)
        plt.show()
else:
    print(f"Showing results failed... Status: {main_job.status()}")
    print("Please check the job logs above or on the OpenEO platform for details.")